In [ ]:
# hide
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
from torch import nn

# model_constructor

> Constructor to create pytorch model. 

## Install

`pip install model-constructor`

Or install from repo:

`pip install git+https://github.com/ayasyrev/model_constructor.git`

## How to use

First import constructor class, then create model constructor object.

Now you can change every part of model.

In [ ]:
from model_constructor import ModelConstructor

In [ ]:
mc = ModelConstructor()

Check base parameters:

In [ ]:
mc

ModelConstructor
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: ReLU, sa: None, se: False
  stem sizes: [64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

Check all parameters with `print_cfg` method:

In [ ]:
mc.print_cfg()

ModelConstructor(
  in_chans=3
  num_classes=1000
  block='BasicBlock'
  conv_layer='ConvBnAct'
  block_sizes=[64, 128, 256, 512]
  layers=[2, 2, 2, 2]
  norm='BatchNorm2d'
  act_fn='ReLU'
  expansion=1
  groups=1
  bn_1st=True
  zero_bn=True
  stem_sizes=[64]
  stem_pool="MaxPool2d {'kernel_size': 3, 'stride': 2, 'padding': 1}"
  init_cnn='init_cnn'
  make_stem='make_stem'
  make_layer='make_layer'
  make_body='make_body'
  make_head='make_head')


Now we have model constructor, default setting as resnet18. And we can get model after call it.

In [ ]:
# collapse_output
model = mc()
model

ModelConstructor(
  (stem): Sequential(
    (conv_1): ConvBnAct(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): BasicBlock(
        (convs): Sequential(
          (conv_0): ConvBnAct(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_fn): ReLU(inplace=True)
          )
          (conv_1): ConvBnAct(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

If you want to change model, just change constructor parameters.  
Lets create resnet34.

In [ ]:
mc.layers = [3, 4, 6, 3]

We can check, what we changed (compare to default constructor).

In [ ]:
mc.changed_fields

{'layers': [3, 4, 6, 3]}

In [ ]:
mc.print_changed_fields()

Changed fields:
layers: [3, 4, 6, 3]


We can compare changed with defaults.

In [ ]:
mc.print_changed_fields(show_default=True)

Changed fields:
layers: [3, 4, 6, 3] | [2, 2, 2, 2]


Now we can look at model parts - stem, body, head.  

In [ ]:
# collapse_output
mc.body

Sequential(
  (l_0): Sequential(
    (bl_0): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): ReLU(inplace=True)
    )
    (bl_1): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv)

## Create constructor from config.

Alternative we can create config first and than create constructor from it. 

In [ ]:
from model_constructor import ModelCfg

In [ ]:
cfg = ModelCfg(
    num_classes=10,
    act_fn=nn.Mish,
)
print(cfg)

ModelCfg(
  in_chans=3
  num_classes=10
  block='BasicBlock'
  conv_layer='ConvBnAct'
  block_sizes=[64, 128, 256, 512]
  layers=[2, 2, 2, 2]
  norm='BatchNorm2d'
  act_fn='Mish'
  expansion=1
  groups=1
  bn_1st=True
  zero_bn=True
  stem_sizes=[64]
  stem_pool="MaxPool2d {'kernel_size': 3, 'stride': 2, 'padding': 1}")


When creating config or constructor we can use string annotation for nn.Modules - it useful when creating model from config files.

In [ ]:
cfg = ModelCfg(
    num_classes=10,
    act_fn="nn.SELU",
)
print(cfg.act_fn)

<class 'torch.nn.modules.activation.SELU'>


Now we can create constructor from config:

In [ ]:
mc = ModelConstructor.from_cfg(cfg)
mc

ModelConstructor
  in_chans: 3, num_classes: 10
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: SELU, sa: False, se: False
  stem sizes: [64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

## More modification.

Main purpose of this module - fast and easy modify model.
And here is the link to more modification to beat Imagenette leaderboard with add MaxBlurPool and modification to ResBlock [notebook](https://github.com/ayasyrev/imagenette_experiments/blob/master/ResnetTrick_create_model_fit.ipynb)  

But now lets create model as mxresnet50 from [fastai forums tread](https://forums.fast.ai/t/how-we-beat-the-5-epoch-imagewoof-leaderboard-score-some-new-techniques-to-consider)  


Lets create mxresnet constructor.

In [ ]:
# mc = ModelConstructor(name="MxResNet")
mc = ModelConstructor()
mc.name = "MxResNet"

Then lets modify stem.

In [ ]:
from model_constructor.xresnet import xresnet_stem

In [ ]:
mc.make_stem = xresnet_stem
mc.stem_sizes = [3, 32, 64, 64]

Now lets change activation function to Mish.
Here is link to [forum discussion](https://forums.fast.ai/t/meet-mish-new-activation-function-possible-successor-to-relu)    
We'v got Mish is in model_constructor.activations, but from pytorch 1.9 take it from torch:

In [ ]:
from torch.nn import Mish

In [ ]:
mc.act_fn = Mish

In [ ]:
mc

MxResNet
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
mc.print_changed_fields()

Changed fields:
name: MxResNet
act_fn: Mish
stem_sizes: [3, 32, 64, 64]
make_stem: xresnet_stem


Here is model:  

In [ ]:
# collapse_output
model = mc()
model

MxResNet(
  act_fn: Mish, stem_sizes: [3, 32, 64, 64], make_stem: xresnet_stem
  (stem): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish(inplace=True)
    )
    (conv_3): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-0

## MXResNet50

Now lets make MxResNet50

In [ ]:
from model_constructor.blocks import BottleneckBlock

In [ ]:
mc.block = BottleneckBlock
mc.expansion = 4
mc.layers = [3, 4, 6, 3]
mc.name = "mxresnet50"

In [ ]:
mc.print_changed_fields()

Changed fields:
name: mxresnet50
block: BottleneckBlock
layers: [3, 4, 6, 3]
act_fn: Mish
expansion: 4
stem_sizes: [3, 32, 64, 64]
stem_pool: MaxPool2d {'kernel_size': 3, 'stride': 2, 'padding': 1}
make_stem: xresnet_stem
make_layer: make_layer
make_body: make_body
make_head: make_head


Now we have mxresnet50 constructor.  
We can inspect every parts of it.  
And after call it we got model.

In [ ]:
mc

mxresnet50
  in_chans: 3, num_classes: 1000
  block: BottleneckBlock,,
  expansion: 4, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

In [ ]:
# collapse_output
mc.stem.conv_1

ConvBnAct(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): Mish(inplace=True)
)

In [ ]:
# collapse_output
mc.body.l_0.bl_0

BasicBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): Mish(inplace=True)
)

We can get model direct way:

In [ ]:
mc = ModelConstructor(
    name="MxResNet",
    act_fn=Mish,
    layers=[3, 4, 6, 3],
    expansion=4,
    make_stem=xresnet_stem,
    stem_sizes=[32, 64, 64],
)
model = mc()

Another way:

In [ ]:
model = ModelConstructor.create_model(
    name="MxResNet",
    act_fn=Mish,
    layers=[3, 4, 6, 3],
    expansion=4,
    make_stem=xresnet_stem,
    stem_sizes=[32, 64, 64],
)

## YaResNet

Now lets change Resblock to YaResBlock (Yet another ResNet, former NewResBlock) is in lib from version 0.1.0

In [ ]:
from model_constructor.yaresnet import YaBasicBlock

In [ ]:
mc = ModelConstructor(name="YaResNet")
mc.block = YaBasicBlock

Or in one line:

In [ ]:
mc = ModelConstructor(name="YaResNet", block=YaBasicBlock)

That all. Now we have YaResNet constructor

In [ ]:
# collapse_output
mc.print_cfg()

ModelConstructor(
  name='YaResNet'
  in_chans=3
  num_classes=1000
  block='YaBasicBlock'
  conv_layer='ConvBnAct'
  block_sizes=[64, 128, 256, 512]
  layers=[2, 2, 2, 2]
  norm='BatchNorm2d'
  act_fn='ReLU'
  expansion=1
  groups=1
  bn_1st=True
  zero_bn=True
  stem_sizes=[64]
  stem_pool="MaxPool2d {'kernel_size': 3, 'stride': 2, 'padding': 1}"
  init_cnn='init_cnn'
  make_stem='make_stem'
  make_layer='make_layer'
  make_body='make_body'
  make_head='make_head')


Let see what we have.

In [ ]:
# collapse_output
mc.body.l_1.bl_0

YaBasicBlock(
  (reduce): ConvBnAct(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): ConvBnAct(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): ReLU(inplace=True)
)

Lets create `xResnet34` like model constructor:

In [ ]:
from model_constructor.helpers import nnModule, MakeModule


class McYaResnet34(ModelConstructor):
    block: nnModule = YaBasicBlock  # nnModule is typing alias, here instead of type[nn.Module]
    layers: list[int] = [3, 4, 6, 3]
    make_stem: MakeModule = xresnet_stem

In [ ]:
mc = McYaResnet34()
mc.print_cfg()

McYaResnet34(
  in_chans=3
  num_classes=1000
  block='YaBasicBlock'
  conv_layer='ConvBnAct'
  block_sizes=[64, 128, 256, 512]
  layers=[3, 4, 6, 3]
  norm='BatchNorm2d'
  act_fn='ReLU'
  expansion=1
  groups=1
  bn_1st=True
  zero_bn=True
  stem_sizes=[64]
  stem_pool="MaxPool2d {'kernel_size': 3, 'stride': 2, 'padding': 1}"
  init_cnn='init_cnn'
  make_stem='xresnet_stem'
  make_layer='make_layer'
  make_body='make_body'
  make_head='make_head')


And `xResnet50` like model can be inherited from `YaResnet34`:

In [ ]:
from model_constructor.yaresnet import YaBottleneckBlock

class McYaResnet50(McYaResnet34):
    block: nnModule = YaBottleneckBlock
    expansion: int = 4

In [ ]:
mc = McYaResnet50()
mc

McYaResnet50
  in_chans: 3, num_classes: 1000
  block: YaBottleneckBlock,
  expansion: 4, groups: 1, dw: False, div_groups: None
  act_fn: ReLU, sa: False, se: False
  stem sizes: [64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

Now we can create model from constructor:

In [ ]:
model = McYaResnet50.create_model()